In [42]:
%matplotlib inline

In [32]:
from cbrain.imports import *
from cbrain.utils import *
from cbrain.data_generator import *
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

In [2]:
limit_mem()

In [3]:
DATA_DIR = '/scratch/srasp/preprocessed_data/'

In [94]:
nlat=10; nlon=30; ngeo=nlat*nlon

In [4]:
features = xr.open_dataset(f'{DATA_DIR}paper_nonorm_train_10days_features.nc')
targets = xr.open_dataset(f'{DATA_DIR}paper_nonorm_train_10days_targets.nc')

In [12]:
norm = xr.open_dataset(f'{DATA_DIR}paper_nonorm_train_fullyear_norm.nc')

In [81]:
x_all = features['features'].values
y_all = targets['targets'].values

In [95]:
x = x_all.reshape(-1, 64, 128, 124)[:, 27:27+nlat, :nlon]
y = y_all.reshape(-1, 64, 128, 120)[:, 27:27+nlat, :nlon]

In [96]:
x = (x - norm['feature_means'].values) / norm['feature_stds'].values
y = y * norm['target_conv'].values

In [97]:
x.shape, y.shape

((480, 10, 30, 124), (480, 10, 30, 120))

## Standard model

In [104]:
x_flat, y_flat = x.reshape(-1, 124), y.reshape(-1, 120)

In [211]:
fc_model = Sequential([
    Dense(1300, input_shape=(124,), activation='relu'),
    Dense(120)
])

In [212]:
fc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 1300)              162500    
_________________________________________________________________
dense_38 (Dense)             (None, 120)               156120    
Total params: 318,620
Trainable params: 318,620
Non-trainable params: 0
_________________________________________________________________


In [213]:
fc_model.compile(Adam(1e-3), 'mse')

In [214]:
fc_model.fit(x_trn[:,:,1], y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val[:,:,1], y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 4s 76us/step - loss: 0.0216 - val_loss: 0.0087
Epoch 2/3
57600/57600 [==============================] - 2s 40us/step - loss: 0.0086 - val_loss: 0.0073
Epoch 3/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0080 - val_loss: 0.0070


In [215]:
fc_model.optimizer.lr = 1e-4
fc_model.fit(x_trn[:,:,1], y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val[:,:,1], y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 2s 40us/step - loss: 0.0078 - val_loss: 0.0068
Epoch 2/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0076 - val_loss: 0.0067
Epoch 3/3
57600/57600 [==============================] - 2s 40us/step - loss: 0.0075 - val_loss: 0.0066


In [216]:
fc_model.optimizer.lr = 2e-5
fc_model.fit(x_trn[:,:,1], y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val[:,:,1], y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 2s 40us/step - loss: 0.0074 - val_loss: 0.0066
Epoch 2/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0073 - val_loss: 0.0065
Epoch 3/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0073 - val_loss: 0.0065


In [219]:
fc_model.optimizer.lr = 1e-5
fc_model.fit(x_trn[:,:,1], y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val[:,:,1], y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0070 - val_loss: 0.0064
Epoch 2/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0070 - val_loss: 0.0064
Epoch 3/3
57600/57600 [==============================] - 2s 39us/step - loss: 0.0069 - val_loss: 0.0064


## RNN

In [110]:
x_re = x.reshape(-1, ngeo, 124)
y_re = y.reshape(-1, ngeo, 120)
x_re.shape, y_re.shape

((480, 300, 124), (480, 300, 120))

In [111]:
x_re = np.rollaxis(x_re.reshape(-1, 2, ngeo, 124), 1, 4).reshape(-1, 124, 2)
y_re = np.rollaxis(y_re.reshape(-1, 2, ngeo, 120), 1, 4).reshape(-1, 120, 2)
x_re.shape, y_re.shape

((72000, 124, 2), (72000, 120, 2))

In [112]:
y_re = y_re[:, :, 1]

In [220]:
rnn_model = Sequential([
    SimpleRNN(512, input_shape=(124, 2,), unroll=True, activation='relu'),
    Dense(120)
])

In [221]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_9 (SimpleRNN)     (None, 512)               263680    
_________________________________________________________________
dense_39 (Dense)             (None, 120)               61560     
Total params: 325,240
Trainable params: 325,240
Non-trainable params: 0
_________________________________________________________________


In [222]:
rnn_model.compile(Adam(1e-3), 'mse')

In [223]:
rnn_model.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 37s 640us/step - loss: 0.0088 - val_loss: 0.0070
Epoch 2/3
57600/57600 [==============================] - 35s 600us/step - loss: 0.0078 - val_loss: 0.0067
Epoch 3/3
57600/57600 [==============================] - 35s 603us/step - loss: 0.0076 - val_loss: 0.0066


In [224]:
rnn_model.optimizer.lr = 1e-4
rnn_model.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 35s 609us/step - loss: 0.0075 - val_loss: 0.0065
Epoch 2/3
57600/57600 [==============================] - 34s 599us/step - loss: 0.0074 - val_loss: 0.0064
Epoch 3/3
57600/57600 [==============================] - 35s 603us/step - loss: 0.0073 - val_loss: 0.0064


In [226]:
rnn_model.optimizer.lr = 2e-5
rnn_model.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 34s 593us/step - loss: 0.0073 - val_loss: 0.0064
Epoch 2/3
57600/57600 [==============================] - 35s 604us/step - loss: 0.0072 - val_loss: 0.0064
Epoch 3/3
57600/57600 [==============================] - 35s 613us/step - loss: 0.0071 - val_loss: 0.0064


In [227]:
rnn_model.optimizer.lr = 1e-5
rnn_model.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=True, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 35s 612us/step - loss: 0.0071 - val_loss: 0.0064
Epoch 2/3
57600/57600 [==============================] - 35s 614us/step - loss: 0.0070 - val_loss: 0.0064
Epoch 3/3
57600/57600 [==============================] - 35s 605us/step - loss: 0.0070 - val_loss: 0.0064


## Stateful

In [127]:
bs = 300

In [145]:
x_state = x.reshape(-1, ngeo, 124); x_state.shape
y_state = y.reshape(-1, ngeo, 120)

In [147]:
x_state = x_state.reshape(x.shape[0]//2, 2, ngeo, 124); x_state.shape
y_state = y_state.reshape(y.shape[0]//2, 2, ngeo, 120); y_state.shape

(240, 2, 300, 120)

In [148]:
x_state = np.rollaxis(x_state, 2, 1).reshape(-1, 2, 124);
y_state = np.rollaxis(y_state, 2, 1).reshape(-1, 2, 120);

In [158]:
x_state = x_state.reshape(-1, 2, 124);
y_state = y_state.reshape(-1, 2, 120);

In [161]:
x_state.shape, y_state.shape

((72000, 2, 124), (72000, 2, 120))

In [162]:
x_state = np.rollaxis(x_state, 1, 3)
y_state = y_state[:, 1]

In [168]:
x_state.shape, y_state.shape

((72000, 124, 2), (72000, 120))

In [169]:
split = int(x_state.shape[0] * 0.8)

In [170]:
x_trn, x_val = x_state[:split], x_state[split:]
y_trn, y_val = y_state[:split], y_state[split:]

In [202]:
rnn_model_state = Sequential([
    SimpleRNN(512, batch_input_shape=(bs, 124, 2,), stateful=True, unroll=True),
    Dense(120)
])

In [203]:
rnn_model_state.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_8 (SimpleRNN)     (300, 512)                263680    
_________________________________________________________________
dense_32 (Dense)             (300, 120)                61560     
Total params: 325,240
Trainable params: 325,240
Non-trainable params: 0
_________________________________________________________________


In [204]:
rnn_model_state.compile(Adam(1e-3), 'mse')

In [205]:
rnn_model_state.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=False, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 37s 637us/step - loss: 0.0112 - val_loss: 0.0075
Epoch 2/3
57600/57600 [==============================] - 34s 596us/step - loss: 0.0081 - val_loss: 0.0071
Epoch 3/3
57600/57600 [==============================] - 34s 597us/step - loss: 0.0080 - val_loss: 0.0070


In [206]:
rnn_model_state.optimizer.lr = 1e-4
rnn_model_state.fit(x_trn, y_trn, batch_size=bs, epochs=3, shuffle=False, 
                    validation_data = (x_val, y_val))

Train on 57600 samples, validate on 14400 samples
Epoch 1/3
57600/57600 [==============================] - 35s 600us/step - loss: 0.0079 - val_loss: 0.0070
Epoch 2/3
57600/57600 [==============================] - 34s 589us/step - loss: 0.0079 - val_loss: 0.0070
Epoch 3/3
57600/57600 [==============================] - 35s 607us/step - loss: 0.0078 - val_loss: 0.0070


Need same train valid splin in above examples!